In [21]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os.path as op
import pickle

In [28]:
A1=np.empty((0,5),dtype='float32')
U1=np.empty((0,7),dtype='float32')
node=['150','149','147','144','142','140','136','61']
mon=['Apr','Mar','Aug','Jun','Jul','Sep','May','Oct']
for j in node:
  for i in mon:
    inp= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[1,2,3,15,16],low_memory=False)
    out= pd.read_csv('data_gkv/AT510_Node_'+str(j)+'_'+str(i)+'19_OutputFile.csv',usecols=[5,6,7,8,17,18,19],low_memory=False)
    
    inp=np.array(inp,dtype='float32')
    out=np.array(out,dtype='float32')
    
    A1=np.append(A1, inp, axis=0)
    U1=np.append(U1, out, axis=0)

print(A1)
print(U1)


[[1.50000e+02 1.90401e+05 7.25000e+02 2.75500e+01 8.03900e+01]
 [1.50000e+02 1.90401e+05 8.25000e+02 2.75600e+01 8.03300e+01]
 [1.50000e+02 1.90401e+05 9.25000e+02 2.75800e+01 8.02400e+01]
 ...
 [6.10000e+01 1.91020e+05 1.94532e+05 2.93700e+01 7.52100e+01]
 [6.10000e+01 1.91020e+05 1.94632e+05 2.93500e+01 7.52700e+01]
 [6.10000e+01 1.91020e+05 1.94732e+05 2.93400e+01 7.53000e+01]]
[[ 28.     3.   -52.   ...  16.97  19.63  20.06]
 [ 28.    15.   -53.   ...  16.63  19.57  23.06]
 [ 31.    16.   -55.   ...  17.24  19.98  20.24]
 ...
 [ 76.    12.   -76.   ...   3.47   3.95   4.35]
 [ 75.    13.   -76.   ...   3.88   4.33   4.42]
 [ 76.    12.   -75.   ...   3.46   4.07   4.28]]


In [23]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(A1)
Y1=scaler_obj.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

In [24]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=0)

In [14]:
import pickle
loaded_model_fit7 = pickle.load(open("Models_File/lasso.sav", 'rb'))
y_test_pred=loaded_model_fit7.predict(x_test)
y_test_pred

array([[0.00011353, 0.0655373 , 0.00011727, ..., 0.01924863, 0.01232052,
        0.00434154],
       [0.00011353, 0.0655373 , 0.00011727, ..., 0.01924863, 0.01232052,
        0.00434154],
       [0.00011353, 0.0655373 , 0.00011727, ..., 0.01924863, 0.01232052,
        0.00434154],
       ...,
       [0.00011353, 0.0655373 , 0.00011727, ..., 0.01924863, 0.01232052,
        0.00434154],
       [0.00011353, 0.0655373 , 0.00011727, ..., 0.01924863, 0.01232052,
        0.00434154],
       [0.00011353, 0.0655373 , 0.00011727, ..., 0.01924863, 0.01232052,
        0.00434154]], dtype=float32)

In [16]:
from numpy import savetxt
savetxt('ARRAY_DATA/lasso_y_test_pred.csv', y_test_pred[:1001], delimiter=',')

In [7]:
from numpy import loadtxt
data = loadtxt('ARRAY_DATA/linearregression_y_test.csv', delimiter=',')
data

array([[0.00010812, 0.06553657, 0.0001368 , ..., 0.00465421, 0.0032433 ,
        0.00139601],
       [0.00011141, 0.06553698, 0.0001189 , ..., 0.00947999, 0.00574444,
        0.00183423],
       [0.00011193, 0.0655366 , 0.0001198 , ..., 0.03056333, 0.01916761,
        0.00634215],
       ...,
       [0.00011604, 0.06553669, 0.00010876, ..., 0.00435393, 0.00298617,
        0.00102436],
       [0.0001126 , 0.06553665, 0.00011905, ..., 0.01626827, 0.01013897,
        0.00530485],
       [0.00010864, 0.06553695, 0.00012487, ..., 0.00378556, 0.00242517,
        0.00079138]])

In [17]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Model,Sequential,load_model
from keras.layers import Input, Embedding
from keras.layers import Dense, Bidirectional
from keras.layers.recurrent import LSTM
import keras.metrics as metrics
import itertools
from tensorflow.python.keras.utils.data_utils import Sequence
from decimal import Decimal
from keras import backend as K
from keras.layers import Conv1D,MaxPooling1D,Flatten,Dense

In [19]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [29]:
from sklearn.preprocessing import MinMaxScaler
import warnings
scaler_obj=MinMaxScaler()
X1=scaler_obj.fit_transform(A1)
Y1=scaler_obj.fit_transform(U1)

warnings.filterwarnings(action='ignore', category=UserWarning)

X1=X1[:,np.newaxis,:]
Y1=Y1[:,np.newaxis,:]

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.25,random_state=0)

In [31]:
from keras.models import model_from_json
json_file = open('lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("lstm.h5")
print("Loaded model from disk")
loaded_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse',metrics=['accuracy','mse','mae',rmse,coeff_determination])
print(loaded_model.evaluate(x_train, y_train, verbose=0))

Loaded model from disk
[0.05459873378276825, 0.021211879327893257, 0.05459883436560631, 0.05130419135093689, 0.12297993898391724, -2921.440185546875]


In [32]:
y_test_pred=loaded_model.predict(x_test)
y_test_pred

array([[0.        , 0.        , 0.        , ..., 0.0356515 , 0.0092808 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.0199545 , 0.02016377,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.02104074, 0.02649951,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.01814228, 0.02314901,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01899886, 0.01032138,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01395327, 0.02754343,
        0.        ]], dtype=float32)

In [48]:
y_test

array([[[0.00010812, 0.06553657, 0.0001368 , ..., 0.00465421,
         0.0032433 , 0.00139601]],

       [[0.00011141, 0.06553698, 0.0001189 , ..., 0.00947999,
         0.00574444, 0.00183423]],

       [[0.00011193, 0.0655366 , 0.0001198 , ..., 0.03056333,
         0.01916761, 0.00634215]],

       ...,

       [[0.0001126 , 0.06553672, 0.00011279, ..., 0.01996804,
         0.01252907, 0.00401423]],

       [[0.00011253, 0.06553655, 0.00012397, ..., 0.03575373,
         0.02222976, 0.00729255]],

       [[0.000112  , 0.06553673, 0.00011786, ..., 0.00466493,
         0.00372249, 0.0052013 ]]], dtype=float32)

In [51]:
y_test=y_test[:,0]

In [52]:
from numpy import savetxt
savetxt('ARRAY_DATA/lstm_y_test.csv', y_test[:1001], delimiter=',')

In [15]:
t=['Y_Actual']*10
t

['Y_Actual',
 'Y_Actual',
 'Y_Actual',
 'Y_Actual',
 'Y_Actual',
 'Y_Actual',
 'Y_Actual',
 'Y_Actual',
 'Y_Actual',
 'Y_Actual']

In [76]:
l1=list()
l1.append(['Y_Actual']*10)
l1.append(np.round(y_test[:10,0],9))
temp1=np.array(l1).transpose()
x1=list(range(1,11))

chart_data1 = pd.DataFrame(temp1, x1,columns=['Data','y'])

l2=list()
l2.append(['Y_Predicted']*10)
l2.append(np.round(y_test_pred[:10,0],9))
temp2=np.array(l2).transpose()
x2=list(range(11,21))

chart_data2 = pd.DataFrame(temp2, x2,columns=['Data','y'])

frames=[chart_data1,chart_data2]

results=pd.concat(frames)
results['x']=np.tile(np.arange(1,11,1),2)
results

,Data,y,x
1,Y_Actual,0.000108116,1
2,Y_Actual,0.000111406,2
3,Y_Actual,0.000111929,3
4,Y_Actual,0.000116041,4
5,Y_Actual,0.000111181,5
6,Y_Actual,0.000109536,6
7,Y_Actual,0.000112378,7
8,Y_Actual,0.000108938,8
9,Y_Actual,0.000112751,9
10,Y_Actual,0.000112751,10


In [77]:
source = results.reset_index().melt('x', var_name='category', value_name='y')

# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['x'], empty='none')

# The basic line
line = alt.Chart(source).mark_line(interpolate='basis').encode(
    x='x',
    y='y',
    color='category:N'
)

# Transparent selectors across the chart. This is what tells us
# the x-value of the cursor
selectors = alt.Chart(source).mark_point().encode(
    x='x',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = line.mark_text(align='left', dx=5, dy=-5).encode(
    text=alt.condition(nearest, 'y', alt.value(' '))
)

# Draw a rule at the location of the selection
rules = alt.Chart(source).mark_rule(color='gray').encode(
    x='x',
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    line, selectors, points, rules, text
).properties(
    width=600, height=300
)

C:\Users\ritik\AppData\Local\Temp/ipykernel_9580/3459456130.py:1: FutureWarning: This dataframe has a column name that matches the 'value_name' column name of the resulting Dataframe. In the future this will raise an error, please set the 'value_name' parameter of DataFrame.melt to a unique name.
  source = results.reset_index().melt('x', var_name='category', value_name='y')


alt.LayerChart(...)

In [51]:
np.arange(11,21,1)

array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [81]:
import altair as alt
from vega_datasets import data

nearest = alt.selection(type='single', nearest=True, on='mouseover',fields=[''], empty='none')
chart=alt.Chart(results).mark_line().encode(
    x='y',
    y='x',
    color='Data',
    strokeDash='Data',
).add_selection(
    nearest
)
chart

alt.Chart(...)

In [48]:
from vega_datasets import data

data.stocks().head(100)

,symbol,date,price
0,MSFT,2000-01-01,39.81
1,MSFT,2000-02-01,36.35
2,MSFT,2000-03-01,43.22
3,MSFT,2000-04-01,28.37
4,MSFT,2000-05-01,25.45
...,...,...,...
95,MSFT,2007-12-01,34.00
96,MSFT,2008-01-01,31.13
97,MSFT,2008-02-01,26.07
98,MSFT,2008-03-01,27.21
